In [9]:
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops.rnn_cell import _linear
import numpy as np
from functools import reduce
from operator import mul

In [10]:
#parameters
d=100
N=60

def flatten(tensor, keep):
    fixed_shape = tensor.get_shape().as_list()
    start = len(fixed_shape) - keep
    left = reduce(mul, [fixed_shape[i] or tf.shape(tensor)[i] for i in range(start)])
    out_shape = [left] + [fixed_shape[i] or tf.shape(tensor)[i] for i in range(start, len(fixed_shape))]
    flat = tf.reshape(tensor, out_shape)
    return flat

def reconstruct(tensor, ref, keep):
    ref_shape = ref.get_shape().as_list()
    tensor_shape = tensor.get_shape().as_list()
    ref_stop = len(ref_shape) - keep
    tensor_start = len(tensor_shape) - keep
    pre_shape = [ref_shape[i] or tf.shape(ref)[i] for i in range(ref_stop)]
    keep_shape = [tensor_shape[i] or tf.shape(tensor)[i] for i in range(tensor_start, len(tensor_shape))]
    # pre_shape = [tf.shape(ref)[i] for i in range(len(ref.get_shape().as_list()[:-keep]))]
    # keep_shape = tensor.get_shape().as_list()[-keep:]
    target_shape = pre_shape + keep_shape
    out = tf.reshape(tensor, target_shape)
    return out

In [11]:
cell=tf.nn.rnn_cell.BasicLSTMCell(d,state_is_tuple=True);
q_mask = tf.placeholder('bool', [N, None], name='q_mask')
x_mask = tf.placeholder('bool', [N, None, None], name='x_mask')
xx=tf.Variable(np.ones([60, 1, 400, 200]),name='xx')
qq=tf.Variable(np.ones([60, 30, 200]),name='qq')

x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 2)  # [N, M]
q_len = tf.reduce_sum(tf.cast(q_mask, 'int32'), 1)  # [N]

flat_inputs_q = flatten(qq, 2)  
flat_len_q = None if q_len is None else tf.cast(flatten(q_len, 0), 'int64')

flat_inputs_x = flatten(xx, 2)  
flat_len_x = None if x_len is None else tf.cast(flatten(x_len, 0), 'int64')

In [12]:
with tf.variable_scope("preprosessing"):
            (flat_fwu_outputs, flat_bwu_outputs), final_state =tf.nn.bidirectional_dynamic_rnn(cell, cell, flat_inputs_q, sequence_length=flat_len_q,dtype='float64',scope='lstm_query')
            fw_u = reconstruct(flat_fwu_outputs, qq, 2)
            bw_u = reconstruct(flat_bwu_outputs, qq, 2)
        
            u = tf.concat([fw_u, bw_u],2)
        
            tf.get_variable_scope().reuse_variables()
            
            (flat_fwh_outputs, flat_bwh_outputs), final_state =tf.nn.bidirectional_dynamic_rnn(cell, cell, flat_inputs_x, sequence_length=flat_len_x, dtype='float64', scope='lstm_context')
            fw_h = reconstruct(flat_fwh_outputs, xx, 2)
            bw_h = reconstruct(flat_bwh_outputs, xx, 2)
            
            h = tf.concat([fw_h, bw_h],3)  # [N, M, JX, 2d]